<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# Step 1: Install required packages (you can run these in your terminal if they are not installed)
!pip3 install beautifulsoup4
!pip3 install requests

# Step 2: Import necessary libraries
import requests
from bs4 import BeautifulSoup

# Step 3: Define the static URL and headers
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# Step 4: Send an HTTP GET request to fetch the page content
response = requests.get(static_url, headers=headers)

# Step 5: Create a BeautifulSoup object from the HTML response
soup = BeautifulSoup(response.text, "html.parser")

# Step 6: Print the page title to verify the page was loaded correctly
print("Page Title:", soup.title.string)



Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Step 5: Create a BeautifulSoup object from the HTML response
soup = BeautifulSoup(response.text, "html.parser")


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [ ]:
# Use soup.title attribute

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [7]:
# Step 1: Use the find_all function in the BeautifulSoup object to find all tables on the page
html_tables = soup.find_all('table')

# Step 2: Print the number of tables found (optional check)
print(f"Found {len(html_tables)} tables on the page.")


Found 25 tables on the page.


Starting from the third table is our target table contains the actual launch records.


In [8]:
# Step 1: Access the third table in the list (index 2)
first_launch_table = html_tables[2]

# Step 2: Print the entire content of the third table to verify its structure
print(first_launch_table)

# Step 3: Extract the column names (headers) from the <th> elements
header_cells = first_launch_table.find_all('th')

# Step 4: Clean and extract the text from each header cell
column_names = [cell.get_text(strip=True) for cell in header_cells]

# Step 5: Print the column names to verify
print("Column Names:", column_names)


<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [9]:
# Initialize an empty list to store column names
column_names = []

# Step 1: Apply find_all() to get all <th> elements in the third table
header_cells = first_launch_table.find_all('th')

# Step 2: Iterate through each <th> element and apply extract_column_from_header() to get the column name
for th in header_cells:
    column_name = extract_column_from_header(th)  # Apply the function to extract the column name
    
    # Step 3: Append the column name to the list only if it is non-empty
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Step 4: Print the extracted column names
print("Extracted Column Names:", column_names)


Extracted Column Names: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [10]:
# Initialize an empty list to store column names
column_names = []

# Apply find_all() to get all <th> elements in the third table
header_cells = first_launch_table.find_all('th')

# Iterate through each <th> element and apply extract_column_from_header() to get the column name
for th in header_cells:
    column_name = extract_column_from_header(th)  # Apply the function to extract the column name
    
    # Append the column name to the list only if it is non-empty
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Print the extracted column names
print("Extracted Column Names:", column_names)


Extracted Column Names: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [ ]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [1]:
import requests
import pandas as pd

# Step 1: Send GET request to the SpaceX API
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)
data = response.json()

# Step 2: Convert the JSON response into a DataFrame
df = pd.json_normalize(data)

# Step 3: Extract the 'static_fire_date_utc' value from the first row
static_fire_date = df.loc[0, 'static_fire_date_utc']

# Step 4: Extract the year
year = pd.to_datetime(static_fire_date).year

print(year)


2006


In [5]:
import requests
import pandas as pd

# Step 1: Fetch all launches data from the SpaceX API
launch_url = 'https://api.spacexdata.com/v4/launches'
launch_response = requests.get(launch_url)
launch_data = launch_response.json()

# Step 2: Fetch rocket details to map IDs to rocket names
rockets_url = 'https://api.spacexdata.com/v4/rockets'
rockets_response = requests.get(rockets_url)
rockets_data = rockets_response.json()

# Step 3: Create a dictionary to map rocket IDs to rocket names
rocket_id_to_name = {rocket['id']: rocket['name'] for rocket in rockets_data}

# Step 4: Convert launch data to DataFrame
df = pd.json_normalize(launch_data)

# Step 5: Add a new column for rocket names using the mapping dictionary
df['rocket_name'] = df['rocket'].map(rocket_id_to_name)

# Step 6: Filter for Falcon 9 launches
falcon_9_launches = df[df['rocket_name'].str.contains('Falcon 9')]

# Step 7: Count Falcon 9 launches
falcon_9_count = falcon_9_launches.shape[0]

print(f"Number of Falcon 9 launches: {falcon_9_count}")


Number of Falcon 9 launches: 195


In [7]:
# Step 1: Check the column names in the DataFrame
print(df.columns)


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


In [8]:
# Step 1: Count missing values in the 'launchpad' column
missing_values_launchpad = df['launchpad'].isna().sum()

# Step 2: Print the result
print(f"Number of missing values in 'launchpad' column: {missing_values_launchpad}")


Number of missing values in 'launchpad' column: 0


In [12]:
import requests
import pandas as pd

# Step 1: Fetch all launches data from the SpaceX API
launch_url = 'https://api.spacexdata.com/v4/launches'
launch_response = requests.get(launch_url)
launch_data = launch_response.json()

# Step 2: Fetch rocket details to map IDs to rocket names
rockets_url = 'https://api.spacexdata.com/v4/rockets'
rockets_response = requests.get(rockets_url)
rockets_data = rockets_response.json()

# Step 3: Create a dictionary to map rocket IDs to names and filter for Falcon 1 and Falcon 9 IDs
rocket_id_to_name = {rocket['id']: rocket['name'] for rocket in rockets_data}

# Filter to get Falcon 1 and Falcon 9 rocket IDs
falcon_1_id = [rocket['id'] for rocket in rockets_data if 'Falcon 1' in rocket['name']]
falcon_9_id = [rocket['id'] for rocket in rockets_data if 'Falcon 9' in rocket['name']]

# Step 4: Convert launch data to DataFrame
df = pd.json_normalize(launch_data)

# Step 5: Filter out Falcon 1 launches by rocket ID and only count Falcon 9 launches
df_filtered = df[df['rocket'].isin(falcon_9_id)]  # Keep only Falcon 9 launches

# Step 6: Count Falcon 9 launches
falcon_9_count = df_filtered.shape[0]

# Step 7: Print the result
print(f"Number of Falcon 9 launches after removing Falcon 1 launches: {falcon_9_count}")


Number of Falcon 9 launches after removing Falcon 1 launches: 195


In [11]:
# Step 1: Initialize an empty dictionary for the launch records
launch_dict = dict.fromkeys(column_names)

# Step 2: Remove the irrelevant column (Date and time ( ) - column was not necessary)
del launch_dict['Date and time ( )']

# Step 3: Initialize the relevant columns with empty lists
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Added some new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Step 4: Initialize row extraction counter
extracted_row = 0

# Step 5: Start iterating through the HTML tables
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Step 6: Iterate through each row in the table
    for rows in table.find_all("tr"):
        # Check if the first table heading is a number corresponding to the launch number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()  # Check if flight number is a digit
        else:
            flag = False  # If there's no flight number, skip this row

        # Get table cells (<td>) if the row is a valid launch record
        row = rows.find_all('td')

        if flag:  # Process only rows with valid flight number
            extracted_row += 1
            
            # Flight Number
            launch_dict['Flight No.'].append(flight_number)

            # Date and Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Booster Version
            bv = booster_version(row[1])
            if not bv:  # If no version found, use the text from <a> tag
                bv = row[1].a.string if row[1].a else "N/A"
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string if row[2].a else "N/A"
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else "N/A"
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else "N/A"
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else "N/A"
            launch_dict['Customer'].append(customer)

            # Launch Outcome
            launch_outcome = list(row[7].strings)[0] if row[7] else "N/A"
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Step 7: Convert the launch_dict into a pandas DataFrame
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

# Step 8: Export the dataframe to a CSV file
df.to_csv('spacex_web_scraped.csv', index=False)

# Optional: Print the first few rows to verify the result
print(df.head())


  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1          2       CCAFS                                Dragon            0   
2          3       CCAFS                                Dragon       525 kg   
3          4       CCAFS                          SpaceX CRS-1     4,700 kg   
4          5       CCAFS                          SpaceX CRS-2     4,877 kg   

  Orbit Customer Launch outcome   Version Booster Booster landing  \
0   LEO   SpaceX      Success\n  F9 v1.07B0003.18         Failure   
1   LEO     NASA        Success  F9 v1.07B0004.18         Failure   
2   LEO     NASA        Success  F9 v1.07B0005.18    No attempt\n   
3   LEO     NASA      Success\n  F9 v1.07B0006.18      No attempt   
4   LEO     NASA      Success\n  F9 v1.07B0007.18    No attempt\n   

              Date   Time  
0      4 June 2010  18:45  
1  8 December 2010  15:43  
2      22 May 2012  07:44 

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
